# Downloading data from the SINAN database

In [1]:
from pysus.online_data import SINAN
import pandas as pd

SINAN is a database of reported cases of certain diseases that Brazilian law requires to be reported. Unfortunately the data available for free download, corresponds only to the investigated cases not the totality of the reported cases. Nevertheless it's an interesting dataset.

To find out what are these diseases, we can use PySUS:

In [2]:
SINAN.list_diseases()

['Animais Peçonhentos',
 'Botulismo',
 'Cancer',
 'Chagas',
 'Chikungunya',
 'Colera',
 'Coqueluche',
 'Contact Communicable Disease',
 'Acidentes de Trabalho',
 'Dengue',
 'Difteria',
 'Esquistossomose',
 'Febre Amarela',
 'Febre Maculosa',
 'Febre Tifoide',
 'Hanseniase',
 'Hantavirose',
 'Hepatites Virais',
 'Intoxicação Exógena',
 'Leishmaniose Visceral',
 'Leptospirose',
 'Leishmaniose Tegumentar',
 'Malaria',
 'Meningite',
 'Peste',
 'Poliomielite',
 'Raiva Humana',
 'Sífilis Adquirida',
 'Sífilis Congênita',
 'Sífilis em Gestante',
 'Tétano Acidental',
 'Tétano Neonatal',
 'Tuberculose',
 'Violência Domestica',
 'Zika']

These diseases are available in countrywide tables, so if we want to see the cases of `Chagas` disease in the state of Minas Gerais, first we can check which years are available:

In [3]:
SINAN.get_available_years('chagas')

/home/fccoelho/Documentos/Software_projects/PySUS/pysus/online_data/SINAN.py:60: UserWarning: Now SINAN tables are no longer split by state. Returning countrywide years
  warnings.warn("Now SINAN tables are no longer split by state. Returning countrywide years")


['CHAGBR00.dbc',
 'CHAGBR01.dbc',
 'CHAGBR02.dbc',
 'CHAGBR03.dbc',
 'CHAGBR04.dbc',
 'CHAGBR05.dbc',
 'CHAGBR06.dbc',
 'CHAGBR07.dbc',
 'CHAGBR08.dbc',
 'CHAGBR09.dbc',
 'CHAGBR10.dbc',
 'CHAGBR11.dbc',
 'CHAGBR12.dbc',
 'CHAGBR13.dbc',
 'CHAGBR14.dbc',
 'CHAGBR15.dbc',
 'CHAGBR16.dbc',
 'CHAGBR17.dbc',
 'CHAGBR18.dbc',
 'CHAGBR19.dbc']

We can see, that we have data from 2000 until 2019. Now we can download it:

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 99 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TP_NOT      19 non-null     object 
 1   ID_AGRAVO   19 non-null     object 
 2   DT_NOTIFIC  19 non-null     object 
 3   SEM_NOT     19 non-null     object 
 4   NU_ANO      19 non-null     object 
 5   SG_UF_NOT   19 non-null     object 
 6   ID_MUNICIP  19 non-null     object 
 7   ID_REGIONA  19 non-null     object 
 8   ID_UNIDADE  19 non-null     object 
 9   DT_SIN_PRI  19 non-null     object 
 10  SEM_PRI     19 non-null     object 
 11  DT_NASC     19 non-null     object 
 12  NU_IDADE_N  19 non-null     int64  
 13  CS_SEXO     19 non-null     object 
 14  CS_GESTANT  19 non-null     object 
 15  CS_RACA     19 non-null     object 
 16  CS_ESCOL_N  19 non-null     object 
 17  SG_UF       19 non-null     object 
 18  ID_MN_RESI  19 non-null     object 
 19  ID_RG_RESI  19 non-null     obj

## Decoding the age in SINAN tables
In SINAN the age comes encoded. PySUS can decode the age column `NU_IDADE_N` into any of these units: years, months, days, or hours.

In [16]:
from pysus.preprocessing.decoders import decodifica_idade_SINAN
decodifica_idade_SINAN?

Signature:       decodifica_idade_SINAN(*args, **kwargs)
Type:            vectorize
String form:     <numpy.vectorize object at 0x7fb554cc31f0>
File:            /usr/local/lib/python3.8/dist-packages/numpy/__init__.py
Docstring:      
Em tabelas do SINAN frequentemente a idade é representada como um inteiro que precisa ser parseado
para retornar a idade em uma unidade cronológica padrão.
:param unidade: unidade da idade: 'Y': anos, 'M' meses, 'D': dias, 'H': horas
:param idade: inteiro ou sequencia de inteiros codificados.
:return:
Class docstring:
vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False,
          signature=None)

Generalized function class.

Define a vectorized function which takes a nested sequence of objects or
numpy arrays as inputs and returns a single numpy array or a tuple of numpy
arrays. The vectorized function evaluates `pyfunc` over successive tuples
of the input arrays like the python map function, except it uses the
broadcasting rules of numpy.

Let's convert the age to years and save it on a different column.

In [17]:
df['idade_anos'] = decodifica_idade_SINAN(df.NU_IDADE_N, 'Y')
df[['NU_IDADE_N', 'idade_anos']]

,NU_IDADE_N,idade_anos
0,4068,68.0
1,4049,49.0
2,4049,49.0
3,4045,45.0
4,4060,60.0
5,4035,35.0
6,4025,25.0
7,4071,71.0
8,4001,1.0
9,4055,55.0


We can easily convert dates and numerical fields in the dataframe:

In [18]:
for cname in df.columns:
    if cname.startswith('DT_'):
        df[cname] = pd.to_datetime(df[cname])
    elif cname.startswith('ID_'):
        try:
            df[cname] = pd.to_numeric(df[cname])
        except ValueError:
            continue
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 100 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   TP_NOT      19 non-null     object        
 1   ID_AGRAVO   19 non-null     object        
 2   DT_NOTIFIC  19 non-null     datetime64[ns]
 3   SEM_NOT     19 non-null     object        
 4   NU_ANO      19 non-null     object        
 5   SG_UF_NOT   19 non-null     object        
 6   ID_MUNICIP  19 non-null     int64         
 7   ID_REGIONA  19 non-null     int64         
 8   ID_UNIDADE  19 non-null     int64         
 9   DT_SIN_PRI  19 non-null     datetime64[ns]
 10  SEM_PRI     19 non-null     object        
 11  DT_NASC     19 non-null     datetime64[ns]
 12  NU_IDADE_N  19 non-null     int64         
 13  CS_SEXO     19 non-null     object        
 14  CS_GESTANT  19 non-null     object        
 15  CS_RACA     19 non-null     object        
 16  CS_ESCOL_N  19 non-null    

## Saving the Modified data
We can save our dataframe in any format we wish to avoid having to redo this analysis next time. If we want to keep only the data from the state of Minas Gerais we need to filter the table using the UF code `31`.

In [ ]:
df['SG_UF_NOT'] = df.SG_UF_NOT.astype(int)
df[df.SG_UF_NOT==31].to_csv('chagas_SP_2018_mod.csv',sep=';',compression='zip')